# TFM Máster Data Science UAH 2020-2021

### MIGUEL PÉREZ CARO

Este notebook tiene como objetivo juntar todos los archivos individuales generados para la recogida de tweets y eliminar los duplicados qu epueda existir, de forma que se obtenga un set de datos final, sin duplicados y sin preprocesar.

Importar librerías

In [1]:
import os
import glob
import pandas as pd

Establecer drectorio de trabajo

In [2]:
directorio = os.path.dirname(os.getcwd())
mydir = os.path.join(directorio, 'data')
os.chdir(mydir)

Obtener todos los archivos a partir de la extensión csv:

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

Concatenar los archivos en uno solo

In [4]:
#df_raw_dup = pd.concat([pd.read_csv(f) for f in all_filenames])

In [5]:
def concatenate(filenames):
    df_raw_dup = pd.DataFrame()
    for f in filenames:
        if 'American' in f:
            df = pd.read_csv(f)
            df['american'] = 1
            df_raw_dup = pd.concat([df_raw_dup, df])
        
        if 'Delta' in f:
            df = pd.read_csv(f)
            df['delta'] = 1
            df_raw_dup = pd.concat([df_raw_dup, df])
        
        if 'Southwest' in f:
            df = pd.read_csv(f)
            df['southwest'] = 1
            df_raw_dup = pd.concat([df_raw_dup, df])
        
        if 'United' in f or 'united' in f:
            df = pd.read_csv(f)
            df['united'] = 1
            df_raw_dup = pd.concat([df_raw_dup, df])
    
    return df_raw_dup

In [6]:
df_raw_dup = concatenate(all_filenames)

Comprobar el tamaño del dataset

In [9]:
df_raw_dup.shape

(732023, 9)

In [11]:
df_raw_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732023 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              732023 non-null  int64  
 1   created_at      732023 non-null  object 
 2   text            732023 non-null  object 
 3   retweet_count   732023 non-null  int64  
 4   favorite_count  732023 non-null  int64  
 5   southwest       168046 non-null  float64
 6   american        194250 non-null  float64
 7   united          180021 non-null  float64
 8   delta           189706 non-null  float64
dtypes: float64(4), int64(3), object(2)
memory usage: 55.8+ MB


Comprobar los tweets duplicados por id

In [10]:
print(df_raw_dup['id'].duplicated().sum())

165376


Estos duplicados se deben a la metodología de extracción de datos, con 3 posibles escenarios:

- Para la recogida de tweets de una aerolínea, se usan dos formatos, por el username, y por el nombre de la aerolínea. Es posible que un mismo tweet mencione la aerlínea por su username y por su nombre, lo que haría que existiese un duplicado.
- Existe la posibilidad de que un mismo tweet mencione 2,3 o las 4 aerolíneas, por lo que sería recogido en cada set de datos correspondiente a cada aerolínea.
- Finalmente, también puede suceder, debido a los tramos de tiempo de recogida, que un mismo tweet aparezca en dos set de datos que se hayan recogido en diferentes fechas.

En el primer y en el último escenario, la eliminación de todos los duplicados menos uno sería suficiente para arreglar el problema, pero en el segundo no, ya que aparecería solo 1 tweet correspondiente a una de las aerolíneas, pero no tendríamos constancia del resto, por lo que hay que tenerlo en cuenta a la hora de tratar el dato. Para ello, en vez de una elimnación de duplicados, se va a realizar un groupby por el id.

En primer lugar observamos el dataset:

In [13]:
df_raw_dup.head()

,id,created_at,text,retweet_count,favorite_count,southwest,american,united,delta
0,1428307624454365191,2021-08-19 10:47:28,Mom claims Southwest Airlines flight attendant...,0,0,1.0,NaN,NaN,NaN
1,1428307318656147462,2021-08-19 10:46:16,Southwest Airlines joins Sabre providing indus...,0,0,1.0,NaN,NaN,NaN
2,1428306497667321858,2021-08-19 10:43:00,RT @TreStewart_: It’s a wild ride to be in the...,13,0,1.0,NaN,NaN,NaN
3,1428306147262402565,2021-08-19 10:41:36,RT @LiveandLetsFly: Southwest Airlines Delays ...,1,0,1.0,NaN,NaN,NaN
4,1428303608810979331,2021-08-19 10:31:31,RT @garethicke: Every single one of these chil...,460,0,1.0,NaN,NaN,NaN


Rellenar los valores nulos con un 0

In [14]:
df_raw_dup['american'] = df_raw_dup['american'].fillna(0)
df_raw_dup['delta'] = df_raw_dup['delta'].fillna(0)
df_raw_dup['southwest'] = df_raw_dup['southwest'].fillna(0)
df_raw_dup['united'] = df_raw_dup['united'].fillna(0)

Se realiza la operación de groupby

In [16]:
df = df_raw_dup.groupby('id').agg({'created_at':'first', 
                           'text': 'first', 
                           'retweet_count':'max',
                           'favorite_count': 'max',
                           'southwest': 'max',
                           'american': 'max',
                           'united': 'max',
                           'delta': 'max'}).reset_index()

Se comprueba el resultado:

In [17]:
df.shape

(566647, 9)

In [18]:
df.head()

,id,created_at,text,retweet_count,favorite_count,southwest,american,united,delta
0,1419550074791763968,2021-07-26 06:48:06,"RT @JohnRLottJr: ""A United Airlines flight was...",4,0,0.0,0.0,1.0,0.0
1,1419551078027202560,2021-07-26 06:52:05,@airvistara I have a booking from HYD to LAX w...,0,0,0.0,0.0,1.0,0.0
2,1419552199525425156,2021-07-26 06:56:33,US$263 - Cheap flights to West Palm Beach from...,0,0,0.0,0.0,1.0,0.0
3,1419553072699920385,2021-07-26 07:00:01,🔁 #ICYMI | The Bureau d’Enquêtes et Analyses (...,0,1,0.0,0.0,1.0,0.0
4,1419555506251837440,2021-07-26 07:09:41,United Airlines evacuates plane ready for take...,0,0,0.0,0.0,1.0,0.0


In [20]:
print(df['id'].duplicated().sum())

0


Parece que ya se ha obtenido un dataset sin duplicados, por lo que se procede a su almacenamiento.

Cabe destacar que el objetivo de este notebook es únicamente el de juntar los set de datos y guardarlos en un único dataset, por lo que aún queda realizar una parte del preprocesado del dato, que incluso incluirá tratamiento de duplicados, ya que en este caso solo se ha tenido en cuenta el tweet id, es decir, el identificador de cada tweet, pero puede haber muchos tweets cuyo texto sea igual.

Guardar el dataset en sin preprocesar

In [21]:
df.to_csv("df_raw.csv", sep = ',', index=False, 
              columns = ['id','created_at','text','retweet_count','favorite_count', 'southwest', 
                         'american', 'united', 'delta'])

### Referencias

* Glob library documentation: https://docs.python.org/3/library/glob.html
* OS library documentation: https://docs.python.org/3/library/os.path.html
* Post: https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/